## UCL 2021-22 | Players Data

**Дано:** Статистика игр и игроков Лиги чемпионов в сезоне 21-22  
**Задача:** Изучить данные, провести исследовательский анализ, составить символическую сборную на основе статистики.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

### Вратари

In [2]:
gk = pd.read_csv('goalkeeping.csv')
gk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53 entries, 0 to 52
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   serial           53 non-null     int64 
 1   player_name      53 non-null     object
 2   club             53 non-null     object
 3   position         53 non-null     object
 4   saved            53 non-null     int64 
 5   conceded         53 non-null     int64 
 6   saved_penalties  53 non-null     int64 
 7   cleansheets      53 non-null     int64 
 8   punches made     53 non-null     int64 
 9   match_played     53 non-null     int64 
dtypes: int64(7), object(3)
memory usage: 4.3+ KB


В датасете указана информация о 53 голкипере.

In [3]:
gk.sample(3)

,serial,player_name,club,position,saved,conceded,saved_penalties,cleansheets,punches made,match_played
49,50,Onana,Ajax,Goalkeeper,2,2,0,0,0,2
18,19,Ederson,Man. City,Goalkeeper,14,14,0,4,1,11
37,38,M. Kerzhakov,Zenit,Goalkeeper,5,4,0,0,0,2


Описание столбцов:  
**serial** -  порядковый номер в дф  
**player_name** - имя игрока  
**club** - футбольный клуб  
**position** - позиция на поле  
**saved** - спасения ворот  
**conceded** - пропущено голов  
**saved_penalties** - взятые пенальти  
**cleansheets** - сухие матчи(без пропущенных голов)  
**punches made** - мячи отбитые кулаками  
**match_played** - количество сыгранных матчей  

Начнём изучение с количества сыгранных матчей.

In [4]:
print(f'Среднее количество игр {gk["match_played"].mean()}')

Среднее количество игр 4.773584905660377


In [5]:
games = gk[['player_name', 'match_played']]
games.sort_values('match_played', ascending=False).head(10)

,player_name,match_played
0,Courtois,13
17,Alisson Becker,13
1,Rulli,12
18,Ederson,11
2,Vlachodimos,10
5,Oblak,10
22,Neuer,9
33,Mendy,9
28,Handanovič,8
15,Köhn,8


Максимальное количество игр в розыгрыше - 13 у Тибо Куртуа. Он дошёл со своей командной до финала. Сам топ-10 по играем заканчивается на отметке 8 игр.  
Посмотрим долю игроков сыгравших менее 3 игр.

In [6]:
len(games.query('match_played < 3'))

16

16 игроков сыграли 1-2 матче, при среднем значении в 4.7 игр. Их показатели не релевантны для выборки.  
Предположим, что целей EDA адекватным порогом для включения в статистику является 6 матчей. Отфильтруем.

In [7]:
true_gk = gk.query('match_played > 5')
true_gk.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21 entries, 0 to 33
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   serial           21 non-null     int64 
 1   player_name      21 non-null     object
 2   club             21 non-null     object
 3   position         21 non-null     object
 4   saved            21 non-null     int64 
 5   conceded         21 non-null     int64 
 6   saved_penalties  21 non-null     int64 
 7   cleansheets      21 non-null     int64 
 8   punches made     21 non-null     int64 
 9   match_played     21 non-null     int64 
dtypes: int64(7), object(3)
memory usage: 1.8+ KB


Итого 21 вратарь попали в финальную выборку.

In [8]:
print(f'Среднее количество игр {true_gk["match_played"].mean()}')

Среднее количество игр 8.19047619047619


Среднее количество игр более 8. Этого вполне достаточно, чтобы составить свой рейтинг эффективности вратарей.  
Топ по количеству игр у нас есть.  
Теперь определим рейтинг и антирейтинг эффективности. Для этого создадим несколько доп.столбцов.

In [9]:
true_gk.sample(3)

,serial,player_name,club,position,saved,conceded,saved_penalties,cleansheets,punches made,match_played
8,9,De Gea,Man. United,Goalkeeper,19,9,0,1,0,7
33,34,Mendy,Chelsea,Goalkeeper,9,7,0,5,2,9
6,7,Musso,Atalanta,Goalkeeper,21,13,0,1,1,6


In [10]:
true_gk['saved_rate'] = true_gk['saved'] / true_gk['match_played']
true_gk['conceded_rate'] = true_gk['conceded'] / true_gk['match_played']

In [11]:
true_gk.sort_values('saved_rate', ascending=False).head(5)

,serial,player_name,club,position,saved,conceded,saved_penalties,cleansheets,punches made,match_played,saved_rate,conceded_rate
0,1,Courtois,Real Madrid,Goalkeeper,61,14,1,5,4,13,4.692308,1.076923
4,5,Mignolet,Club Brugge,Goalkeeper,28,20,0,0,1,6,4.666667,3.333333
2,3,Vlachodimos,Benfica,Goalkeeper,38,17,1,5,5,10,3.800000,1.700000
6,7,Musso,Atalanta,Goalkeeper,21,13,0,1,1,6,3.500000,2.166667
1,2,Rulli,Villarreal,Goalkeeper,41,16,0,3,12,12,3.416667,1.333333


In [12]:
true_gk.sort_values('saved', ascending=False).head(5)

,serial,player_name,club,position,saved,conceded,saved_penalties,cleansheets,punches made,match_played,saved_rate,conceded_rate
0,1,Courtois,Real Madrid,Goalkeeper,61,14,1,5,4,13,4.692308,1.076923
1,2,Rulli,Villarreal,Goalkeeper,41,16,0,3,12,12,3.416667,1.333333
2,3,Vlachodimos,Benfica,Goalkeeper,38,17,1,5,5,10,3.800000,1.700000
4,5,Mignolet,Club Brugge,Goalkeeper,28,20,0,0,1,6,4.666667,3.333333
5,6,Oblak,Atlético,Goalkeeper,26,10,0,3,2,10,2.600000,1.000000


**Вывод**: Куртуа лучший не только по количеству матчей и колчеству сейвов в целом, но и показателю "количество сейвов на 1 матч".   
**Вывод 2**: Топ по общему количеству сейвов почти не отличается от топа по показателю "количество сейвов на 1 матч".

In [13]:
true_gk.sort_values('saved_rate').head(5)

,serial,player_name,club,position,saved,conceded,saved_penalties,cleansheets,punches made,match_played,saved_rate,conceded_rate
33,34,Mendy,Chelsea,Goalkeeper,9,7,0,5,2,9,1.000000,0.777778
17,18,Alisson Becker,Liverpool,Goalkeeper,15,14,0,4,3,13,1.153846,1.076923
18,19,Ederson,Man. City,Goalkeeper,14,14,0,4,1,11,1.272727,1.272727
28,29,Handanovič,Inter,Goalkeeper,12,7,0,3,5,8,1.500000,0.875000
22,19,Neuer,Bayern,Goalkeeper,14,6,0,4,1,9,1.555556,0.666667


А вот и антирейтинг по сейвам. Худшим является голкипер Челси - Менди. Удивительно, что в антитоп попали такие знаменитости, как Эдерсон, Алиссон, Нойер и Ханданович.  
При том, что Алиссон и Эдерсон являются одними из лидеров по количеству матчей.  

Изучим показатель пропущенных мячей.

In [14]:
true_gk.sort_values('conceded_rate', ascending=False).head(5)

,serial,player_name,club,position,saved,conceded,saved_penalties,cleansheets,punches made,match_played,saved_rate,conceded_rate
4,5,Mignolet,Club Brugge,Goalkeeper,28,20,0,0,1,6,4.666667,3.333333
6,7,Musso,Atalanta,Goalkeeper,21,13,0,1,1,6,3.500000,2.166667
14,15,Adán,Sporting CP,Goalkeeper,17,13,0,2,8,7,2.428571,1.857143
10,11,Diogo Costa,Porto,Goalkeeper,18,11,0,2,2,6,3.000000,1.833333
19,19,Kobel,Dortmund,Goalkeeper,14,11,1,2,1,6,2.333333,1.833333


Голкиперу ФК Брюгге этот розыгрышь ЛЧ принёс максимальное разочарование. Он пропускал в среднем 3,3 мяча за игру.   
У ближайших преследователей в антирейтинге этот показатель составляет от 1.8 до 2.16.  
Изучим расклад по количеству голов.

In [15]:
true_gk.sort_values('conceded', ascending=False).head(5)

,serial,player_name,club,position,saved,conceded,saved_penalties,cleansheets,punches made,match_played,saved_rate,conceded_rate
4,5,Mignolet,Club Brugge,Goalkeeper,28,20,0,0,1,6,4.666667,3.333333
2,3,Vlachodimos,Benfica,Goalkeeper,38,17,1,5,5,10,3.800000,1.700000
1,2,Rulli,Villarreal,Goalkeeper,41,16,0,3,12,12,3.416667,1.333333
0,1,Courtois,Real Madrid,Goalkeeper,61,14,1,5,4,13,4.692308,1.076923
15,16,Köhn,Salzburg,Goalkeeper,16,14,0,1,3,8,2.000000,1.750000


**Вывод**: трое лидеров рейтинга по сейвам оказались в антирейтинге по количеству голов.  Это Courtois, Rulli, Vlachodimos.  
По совокупности(количество сейвов и количество пропущенных) можно сделать вывод о не высокой надежности обороны команд, в которых выступают данные голкиперы.  

Выведем рейтинг малопропускающих команд.

In [16]:
true_gk.sort_values('conceded_rate').head(5)

,serial,player_name,club,position,saved,conceded,saved_penalties,cleansheets,punches made,match_played,saved_rate,conceded_rate
27,25,Grbić,LOSC,Goalkeeper,13,4,0,3,1,6,2.166667,0.666667
22,19,Neuer,Bayern,Goalkeeper,14,6,0,4,1,9,1.555556,0.666667
33,34,Mendy,Chelsea,Goalkeeper,9,7,0,5,2,9,1.000000,0.777778
23,19,Bounou,Sevilla,Goalkeeper,14,5,0,2,1,6,2.333333,0.833333
28,29,Handanovič,Inter,Goalkeeper,12,7,0,3,5,8,1.500000,0.875000


**Вывод**: чуть выше мы определили Нойера, Хандановича и Менди как голкиперов с худишими показателями по сейвам.  
Несмотря на это их команды пропускают меньше всего.  Это говорит о том, что у этих команд высокая эффективность оборонительной линии. Учтём это при изучении оборонительных игроков.

In [17]:
true_gk.sort_values('saved_penalties', ascending=False).head(5)

,serial,player_name,club,position,saved,conceded,saved_penalties,cleansheets,punches made,match_played,saved_rate,conceded_rate
0,1,Courtois,Real Madrid,Goalkeeper,61,14,1,5,4,13,4.692308,1.076923
2,3,Vlachodimos,Benfica,Goalkeeper,38,17,1,5,5,10,3.800000,1.700000
19,19,Kobel,Dortmund,Goalkeeper,14,11,1,2,1,6,2.333333,1.833333
17,18,Alisson Becker,Liverpool,Goalkeeper,15,14,0,4,3,13,1.153846,1.076923
28,29,Handanovič,Inter,Goalkeeper,12,7,0,3,5,8,1.500000,0.875000


Всего 3 пенальти было отбито вратарями за весь розыгрышь. Данные показатель малоинформативен. Изучим показатель по сухим матчам.

In [18]:
true_gk.sort_values('cleansheets', ascending=False).head(5)

,serial,player_name,club,position,saved,conceded,saved_penalties,cleansheets,punches made,match_played,saved_rate,conceded_rate
0,1,Courtois,Real Madrid,Goalkeeper,61,14,1,5,4,13,4.692308,1.076923
2,3,Vlachodimos,Benfica,Goalkeeper,38,17,1,5,5,10,3.800000,1.700000
33,34,Mendy,Chelsea,Goalkeeper,9,7,0,5,2,9,1.000000,0.777778
18,19,Ederson,Man. City,Goalkeeper,14,14,0,4,1,11,1.272727,1.272727
17,18,Alisson Becker,Liverpool,Goalkeeper,15,14,0,4,3,13,1.153846,1.076923


По сухим матчам нужно выделить Vlachodimos и Courtois. С учётом показателя по сейвам, можно сделать вывод, что их вклад в отсутствие голов наиболее весомый среди всего топа.

**Вывод**: по результатам исследования, наиболее справедливыми кандидатами среди вратарей являются:
1. Courtois	
2. Vlachodimos

### Защитные действия

In [19]:
defend = pd.read_csv('defending.csv')
defend.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 627 entries, 0 to 626
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   serial               627 non-null    int64 
 1   player_name          627 non-null    object
 2   club                 627 non-null    object
 3   position             627 non-null    object
 4   balls_recoverd       627 non-null    int64 
 5   tackles              627 non-null    int64 
 6   t_won                627 non-null    int64 
 7   t_lost               627 non-null    int64 
 8   clearance_attempted  627 non-null    int64 
 9   match_played         627 non-null    int64 
dtypes: int64(7), object(3)
memory usage: 49.1+ KB


In [20]:
defend.sample(3)

,serial,player_name,club,position,balls_recoverd,tackles,t_won,t_lost,clearance_attempted,match_played
625,578,Zakaria,Juventus,Midfielder,1,0,0,0,0,1
83,82,Tolói,Atalanta,Defender,34,6,5,1,6,4
79,76,Giménez,Atlético,Defender,35,4,2,2,23,7


Описание столбцов:  
serial - порядковый номер в дф  
player_name - имя игрока  
club - футбольный клуб  
position - позиция на поле  
balls_recoverd - отбор  
tackles	- единоборство  
t_won - успешное ед.  
t_lost - безуспешное ед.  
clearance_attempted - вынос мяча(снижение угрозы на ворота)  
match_played - сыграно игр  

Начнём изучение с количества сыгранных матчей.

In [21]:
print(f'Среднее количество игр {defend["match_played"].mean()}')

Среднее количество игр 5.4720893141945774


In [22]:
games = defend[['player_name', 'match_played']]
games.sort_values('match_played', ascending=False).head(10)

,player_name,match_played
123,Vinícius Júnior,13
368,Mané,13
242,Luis Díaz,13
315,Salah,13
3,Fabinho,13
56,Modrić,13
4,Albiol,12
414,Sterling,12
27,Parejo,12
200,Mahrez,12


In [23]:
true_def = defend.query('match_played > 5')
true_def.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 296 entries, 0 to 600
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   serial               296 non-null    int64 
 1   player_name          296 non-null    object
 2   club                 296 non-null    object
 3   position             296 non-null    object
 4   balls_recoverd       296 non-null    int64 
 5   tackles              296 non-null    int64 
 6   t_won                296 non-null    int64 
 7   t_lost               296 non-null    int64 
 8   clearance_attempted  296 non-null    int64 
 9   match_played         296 non-null    int64 
dtypes: int64(7), object(3)
memory usage: 25.4+ KB


Благодаря фильтру по количеству матчей в выборка похудела более чем на половину.  
В дф присутсвуют 3 показателя, которые нам нужны для оценки оборонительной эффективности:   
- отоборы
- доля успешных единоборств
- выносы 

In [24]:
true_def['bc_rate'] = true_def['balls_recoverd'] / true_def['match_played']
true_def['tackles_rate'] =  true_def['t_won'] / true_def['tackles']
true_def['tackles_rate'] =  true_def['tackles_rate'] / true_def['match_played']
true_def['clearance_rate'] = true_def['clearance_attempted'] / true_def['match_played']

In [25]:
true_def.groupby('position')[['bc_rate', 'tackles_rate', 'clearance_rate']].agg(['count', 'mean'])

bc_rate           tackles_rate           clearance_rate          
             count      mean        count      mean          count      mean
position                                                                    
Defender        96  4.633026           94  0.063929             96  2.501623
Forward         67  1.212732           58  0.039759             67  0.302215
Midfielder     133  3.059832          131  0.056248            133  0.685662

Вывели усредненые метрики по каждому типу позиций игроков. Ожидаемо лучшие показатели у игроков оборонительного плана.  
Уберем из выборки нападающих, так как как оборона не является их приоритетным функционалом.

In [26]:
true_def = true_def.query('position != "Forward"')
len(true_def)

229

**ТОП-7 по показателю отборов за 1 матч**

In [39]:
true_def.sort_values('bc_rate', ascending=False).head(7)

,serial,player_name,club,position,balls_recoverd,tackles,t_won,t_lost,clearance_attempted,match_played,bc_rate,tackles_rate,clearance_rate
18,17,Fernando,Sevilla,Midfielder,51,3,1,2,6,6,8.500000,0.055556,1.000000
2,1,Reinildo,Atlético,Defender,76,13,6,7,20,10,7.600000,0.046154,2.000000
5,6,Thiago Silva,Chelsea,Defender,65,9,4,5,17,9,7.222222,0.049383,1.888889
36,37,Lauper,Young Boys,Midfielder,43,12,7,5,13,6,7.166667,0.097222,2.166667
25,22,De Ligt,Juventus,Defender,49,5,4,1,24,7,7.000000,0.114286,3.428571
38,38,Meunier,Dortmund,Defender,42,3,0,3,9,6,7.000000,0.000000,1.500000
21,22,André,LOSC,Midfielder,49,24,12,12,11,7,7.000000,0.071429,1.571429


**ТОП-7 по показателю успешных единоборств за 1 матч**

In [40]:
true_def.sort_values('tackles_rate', ascending=False).head(7)

,serial,player_name,club,position,balls_recoverd,tackles,t_won,t_lost,clearance_attempted,match_played,bc_rate,tackles_rate,clearance_rate
348,337,Bruno,Sheriff,Midfielder,10,2,2,0,6,6,1.666667,0.166667,1.000000
213,205,Aké,Man. City,Defender,19,3,3,0,19,6,3.166667,0.166667,3.166667
239,226,Oxlade-Chamberlain,Liverpool,Midfielder,17,1,1,0,4,6,2.833333,0.166667,0.666667
195,189,Tonali,Milan,Midfielder,21,2,2,0,6,6,3.500000,0.166667,1.000000
493,479,Marlos,Shakhtar Donetsk,Midfielder,4,1,1,0,1,6,0.666667,0.166667,0.166667
80,76,Lacroix,Wolfsburg,Defender,35,1,1,0,30,6,5.833333,0.166667,5.000000
309,300,Dimarco,Inter,Defender,12,2,2,0,7,7,1.714286,0.142857,1.000000


**ТОП-7 по показателю выносов за 1 матч**

In [41]:
true_def.sort_values('clearance_rate', ascending=False).head(7)

,serial,player_name,club,position,balls_recoverd,tackles,t_won,t_lost,clearance_attempted,match_played,bc_rate,tackles_rate,clearance_rate
53,54,Fernando Costanza,Sheriff,Defender,39,19,13,6,39,6,6.500000,0.114035,6.500000
97,95,Dulanto,Sheriff,Defender,32,11,3,8,39,6,5.333333,0.045455,6.500000
119,110,Arboleda,Sheriff,Defender,30,1,0,1,37,6,5.000000,0.000000,6.166667
6,7,Otamendi,Benfica,Defender,63,28,17,11,48,9,7.000000,0.067460,5.333333
4,5,Albiol,Villarreal,Defender,73,9,6,3,62,12,6.083333,0.055556,5.166667
24,22,Pau Torres,Villarreal,Defender,49,7,3,4,62,12,4.083333,0.035714,5.166667
54,54,Wöber,Salzburg,Defender,39,14,6,8,36,7,5.571429,0.061224,5.142857


**Вывод**: по результам изучения оборонительной эффективности для включения в симв.сборную можно предложить следующих кандидатов:  
- Reinildo	 (лучший показатель по отоборам среди защитников)  
- Thiago Silva (второй показатель по отоборам среди защитников, игрок Челси - команды с мин. количество сейвов у голкипера)
- De Ligt (третий показатель по отоборам среди защитников)
- Aké (первый среди защитников по выигранным единоборствам за матч, игрок Сити - команды с мин. количество сейвов у голкипера)

**Вывод 2**: в качестве полузащитников оборонительного плана нужно отметить:  
- Fernando
- Lauper

### Ассистенты

In [42]:
attack = pd.read_csv('attacking.csv')
attack.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176 entries, 0 to 175
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   serial        176 non-null    int64 
 1   player_name   176 non-null    object
 2   club          176 non-null    object
 3   position      176 non-null    object
 4   assists       176 non-null    int64 
 5   corner_taken  176 non-null    int64 
 6   offsides      176 non-null    int64 
 7   dribbles      176 non-null    int64 
 8   match_played  176 non-null    int64 
dtypes: int64(6), object(3)
memory usage: 12.5+ KB


In [43]:
attack.sample(5)

,serial,player_name,club,position,assists,corner_taken,offsides,dribbles,match_played
77,64,Malcom,Zenit,Forward,1,6,2,12,6
19,12,Zapata,Atalanta,Forward,3,0,7,10,6
164,64,Tagliafico,Ajax,Defender,1,0,0,1,3
119,64,Blind,Ajax,Defender,1,0,1,8,8
22,12,Cristiano,Sheriff,Defender,3,0,1,7,6


In [44]:
attack = attack.query('match_played > 5')

In [45]:
attack['assists_rate'] = attack['assists'] / attack['match_played']
attack['dribbles_rate'] = attack['dribbles'] / attack['match_played']

In [46]:
attack.sort_values('assists_rate', ascending=False).head(7)

,serial,player_name,club,position,assists,corner_taken,offsides,dribbles,match_played,assists_rate,dribbles_rate
0,1,Bruno Fernandes,Man. United,Midfielder,7,10,2,7,7,1.000000,1.000000
3,4,Antony,Ajax,Forward,5,3,4,28,7,0.714286,4.000000
2,2,Sané,Bayern,Midfielder,6,3,3,32,10,0.600000,3.200000
9,5,Gerard Moreno,Villarreal,Forward,4,0,3,9,7,0.571429,1.285714
8,5,Mbappé,Paris,Forward,4,4,8,43,8,0.500000,5.375000
22,12,Cristiano,Sheriff,Defender,3,0,1,7,6,0.500000,1.166667
19,12,Zapata,Atalanta,Forward,3,0,7,10,6,0.500000,1.666667


**Вывод**: лучшие игроки по показателю голевых пасов:
- Bruno Fernandes
- Antony
- Sané

In [47]:
attack.sort_values('dribbles_rate', ascending=False).head(7)

,serial,player_name,club,position,assists,corner_taken,offsides,dribbles,match_played,assists_rate,dribbles_rate
16,12,Coman,Bayern,Forward,3,4,4,59,9,0.333333,6.555556
1,2,Vinícius Júnior,Real Madrid,Forward,6,3,4,83,13,0.461538,6.384615
94,64,Moumi Ngamaleu,Young Boys,Midfielder,1,1,0,34,6,0.166667,5.666667
8,5,Mbappé,Paris,Forward,4,4,8,43,8,0.500000,5.375000
73,64,Neymar,Paris,Forward,1,7,1,27,6,0.166667,4.500000
18,12,Bellingham,Dortmund,Midfielder,3,1,1,24,6,0.500000,4.000000
3,4,Antony,Ajax,Forward,5,3,4,28,7,0.714286,4.000000


**Вывод 2:** лучшие дриблеры розыгрыша:  
- Coman	
- Vinícius Júnior
- Mbappé

### Голы, минуты, дистанция

In [48]:
key_s = pd.read_csv('key_stats.csv')
key_s.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 747 entries, 0 to 746
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   player_name       747 non-null    object
 1   club              747 non-null    object
 2   position          747 non-null    object
 3   minutes_played    747 non-null    int64 
 4   match_played      747 non-null    int64 
 5   goals             747 non-null    int64 
 6   assists           747 non-null    int64 
 7   distance_covered  747 non-null    object
dtypes: int64(4), object(4)
memory usage: 46.8+ KB


In [51]:
key_s = key_s.query('match_played > 5')

In [52]:
key_s['distance_covered'] = key_s['distance_covered'].astype(float)

In [53]:
key_s.sample(3)

,player_name,club,position,minutes_played,match_played,goals,assists,distance_covered
225,Nsoki,Club Brugge,Defender,484,6,0,0,56.8
181,Tiago Djaló,LOSC,Defender,533,7,0,0,60.5
411,M. Olsson,Malmö,Defender,284,6,0,0,34.5


In [54]:
key_s = key_s.query('match_played > 5')
len(key_s)

319

In [55]:
key_s['dis_rate'] = key_s['distance_covered'] / key_s['match_played']
key_s['goals_rate'] = key_s['minutes_played'] / key_s['goals']
key_s['assists_rate'] = key_s['minutes_played'] / key_s['assists']

In [56]:
key_s.sample(5)

,player_name,club,position,minutes_played,match_played,goals,assists,distance_covered,dis_rate,goals_rate,assists_rate
283,Kovačić,Chelsea,Midfielder,426,6,0,0,50.3,8.383333,inf,inf
78,Müller,Bayern,Forward,778,10,4,3,98.5,9.850000,194.5,259.333333
355,Okafor,Salzburg,Midfielder,345,6,3,0,40.0,6.666667,115.0,inf
129,Zeki Çelik,LOSC,Defender,591,7,0,1,71.3,10.185714,inf,591.000000
226,Renan Lodi,Atlético,Defender,478,10,1,2,61.7,6.170000,478.0,239.000000


Изучим показатель дистанции за матч по позициям.

In [57]:
key_s.groupby('position')['dis_rate'].agg(['count', 'mean'])

,count,mean
position,,
Defender,96,8.812482
Forward,68,7.588190
Goalkeeper,21,5.227756
Midfielder,134,8.353973


**Вывод**: больше всего за матч в среднем пробегают именно защитники. Неожиданно.

**ТОП-7 по пробегу за матч**

In [58]:
key_s.sort_values('dis_rate', ascending=False).head(7)

,player_name,club,position,minutes_played,match_played,goals,assists,distance_covered,dis_rate,goals_rate,assists_rate
92,Aaronson,Salzburg,Midfielder,715,8,0,2,103.5,12.937500,inf,357.5
153,S. Thill,Sheriff,Midfielder,540,6,2,1,76.8,12.800000,270.0,540.0
74,Weigl,Benfica,Midfielder,810,9,0,0,112.0,12.444444,inf,inf
95,Kimmich,Bayern,Midfielder,702,8,0,0,96.4,12.050000,inf,inf
190,Freuler,Atalanta,Midfielder,526,6,1,1,71.6,11.933333,526.0,526.0
152,Vanaken,Club Brugge,Midfielder,540,6,3,1,69.4,11.566667,180.0,540.0
178,Sydorchuk,Dynamo Kyiv,Midfielder,536,6,0,0,69.1,11.516667,inf,inf


Данный топ подтверждает предыдущий вывод. Все 7ро в топе - защитники.

**ТОП-7 по голам**

In [59]:
key_s.sort_values('goals_rate', ascending=True).head(7)

,player_name,club,position,minutes_played,match_played,goals,assists,distance_covered,dis_rate,goals_rate,assists_rate
102,Haller,Ajax,Forward,668,8,11,1,82.2,10.275000,60.727273,668.0
40,Firmino,Liverpool,Midfielder,323,7,5,0,43.0,6.142857,64.600000,inf
66,Lewandowski,Bayern,Forward,876,10,13,3,99.7,9.970000,67.384615,292.0
2,Benzema,Real Madrid,Forward,1106,12,15,1,121.5,10.125000,73.733333,1106.0
184,Nkunku,Leipzig,Midfielder,531,6,7,0,62.8,10.466667,75.857143,inf
346,Jesus,Man. City,Forward,352,8,4,1,41.9,5.237500,88.000000,352.0
12,Rodrygo,Real Madrid,Forward,505,11,5,2,65.7,5.972727,101.000000,252.5


Самыми забивными футболистами турнира можно считать:
- Haller
- Firmino
- Lewandowski	

Они забивали чаще 1 гола за 70 минут игрового времени.

Посмотрим просто количественный показатель по голам.

In [60]:
key_s.sort_values('goals', ascending=False).head(7)

,player_name,club,position,minutes_played,match_played,goals,assists,distance_covered,dis_rate,goals_rate,assists_rate
2,Benzema,Real Madrid,Forward,1106,12,15,1,121.5,10.125000,73.733333,1106.0
66,Lewandowski,Bayern,Forward,876,10,13,3,99.7,9.970000,67.384615,292.0
102,Haller,Ajax,Forward,668,8,11,1,82.2,10.275000,60.727273,668.0
26,Salah,Liverpool,Forward,1008,13,8,2,112.0,8.615385,126.000000,504.0
59,Mahrez,Man. City,Midfielder,986,12,7,2,120.1,10.008333,140.857143,493.0
184,Nkunku,Leipzig,Midfielder,531,6,7,0,62.8,10.466667,75.857143,inf
76,Sané,Bayern,Midfielder,798,10,6,6,94.0,9.400000,133.000000,133.0


ТОП практически не изменился.
Бензема занимает первое место как лучший бомбарди турнира.  

**ТОП-7 по голевым пасам**

In [61]:
key_s.sort_values('assists_rate', ascending=True).head(7)

,player_name,club,position,minutes_played,match_played,goals,assists,distance_covered,dis_rate,goals_rate,assists_rate
196,Bruno Fernandes,Man. United,Midfielder,520,7,0,7,58.4,8.342857,inf,74.285714
15,Asensio,Real Madrid,Forward,207,8,1,2,28.4,3.550000,207.000000,103.500000
138,Antony,Ajax,Forward,577,7,2,5,65.1,9.300000,288.500000,115.400000
214,João Mário,Benfica,Midfielder,493,8,0,4,66.0,8.250000,inf,123.250000
192,Gerard Moreno,Villarreal,Forward,524,7,2,4,65.7,9.385714,262.000000,131.000000
76,Sané,Bayern,Midfielder,798,10,6,6,94.0,9.400000,133.000000,133.000000
237,Zapata,Atalanta,Forward,464,6,3,3,50.1,8.350000,154.666667,154.666667


И тут Бруно Фернандеш занимает перое место со значительным отрывом от преследователей.  
**Вывод:** по рассмотренным метрикам для символической сборной можно выделить следующих игроков:
- Бензема
- Бруно
- Левандовски
- Сане

### Детализация по голам

In [62]:
goals = pd.read_csv('goals.csv')
goals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183 entries, 0 to 182
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   serial         183 non-null    int64 
 1   player_name    183 non-null    object
 2   club           183 non-null    object
 3   position       183 non-null    object
 4   goals          183 non-null    int64 
 5   right_foot     183 non-null    int64 
 6   left_foot      183 non-null    int64 
 7   headers        183 non-null    int64 
 8   others         183 non-null    int64 
 9   inside_area    183 non-null    int64 
 10  outside_areas  183 non-null    int64 
 11  penalties      183 non-null    int64 
 12  match_played   183 non-null    int64 
dtypes: int64(10), object(3)
memory usage: 18.7+ KB


In [63]:
goals = goals.query('match_played > 5')
len(goals)

129

In [64]:
goals.sample(3)

,serial,player_name,club,position,goals,right_foot,left_foot,headers,others,inside_area,outside_areas,penalties,match_played
80,73,Trigueros,Villarreal,Midfielder,1,1,0,0,0,1,0,0,9
105,73,De Vrij,Inter,Defender,1,1,0,0,0,1,0,0,6
166,73,Origi,Liverpool,Forward,1,0,0,1,0,1,0,0,7


Столбцы таблицы:  
right_foot - голы правой ногой  
left_foot - голы левой ногой  
headers - голы головой  
others - голы другими частями тела
inside_area - голы из штрафной  
outside_areas - голы из за пределов штрафной

In [65]:
goals = goals.query('goals > 4')
len(goals)

15

Всего 15 игроков смогли забить в этом розыгрше более 4 голов. Рассмотрим их достижения детальнее.

In [66]:
goals.sort_values('headers', ascending=False).head(5)

,serial,player_name,club,position,goals,right_foot,left_foot,headers,others,inside_area,outside_areas,penalties,match_played
0,1,Benzema,Real Madrid,Forward,15,11,1,3,0,13,2,3,12
2,3,Haller,Ajax,Forward,11,3,4,3,1,11,0,1,8
4,5,Nkunku,Leipzig,Midfielder,7,3,1,3,0,7,0,0,6
5,5,Mahrez,Man. City,Midfielder,7,1,4,2,0,7,0,2,12
1,2,Lewandowski,Bayern,Forward,13,8,3,1,1,13,0,3,10


**Вывод:** Лучший по голам головой Nkunku - 3 гола за 7 матчей.

In [67]:
goals.sort_values('left_foot', ascending=False).head(5)

,serial,player_name,club,position,goals,right_foot,left_foot,headers,others,inside_area,outside_areas,penalties,match_played
3,4,Salah,Liverpool,Forward,8,0,8,0,0,7,1,1,13
10,7,Sané,Bayern,Midfielder,6,1,5,0,0,3,3,0,10
14,12,Messi,Paris,Forward,5,0,5,0,0,3,2,2,7
2,3,Haller,Ajax,Forward,11,3,4,3,1,11,0,1,8
5,5,Mahrez,Man. City,Midfielder,7,1,4,2,0,7,0,2,12


**Вывод**: Лучший левоногий футболист турнира - Salah

In [68]:
goals.sort_values('right_foot', ascending=False).head(5)

,serial,player_name,club,position,goals,right_foot,left_foot,headers,others,inside_area,outside_areas,penalties,match_played
0,1,Benzema,Real Madrid,Forward,15,11,1,3,0,13,2,3,12
1,2,Lewandowski,Bayern,Forward,13,8,3,1,1,13,0,3,10
6,7,Mbappé,Paris,Forward,6,6,0,0,0,6,0,0,8
7,7,Núñez,Benfica,Forward,6,5,0,1,0,6,0,1,10
8,7,Ronaldo,Man. United,Forward,6,5,0,1,0,4,2,0,7


**Вывод**: Лучшие правоногие футболисты турнира - Benzema, Lewandowski, Mbappé

In [69]:
goals.sort_values('outside_areas', ascending=False).head(5)

,serial,player_name,club,position,goals,right_foot,left_foot,headers,others,inside_area,outside_areas,penalties,match_played
10,7,Sané,Bayern,Midfielder,6,1,5,0,0,3,3,0,10
0,1,Benzema,Real Madrid,Forward,15,11,1,3,0,13,2,3,12
8,7,Ronaldo,Man. United,Forward,6,5,0,1,0,4,2,0,7
14,12,Messi,Paris,Forward,5,0,5,0,0,3,2,2,7
3,4,Salah,Liverpool,Forward,8,0,8,0,0,7,1,1,13


**Вывод**: Лучший футболист забивающий из-за пределов штрафной - Sane

In [70]:
goals.sort_values('inside_area', ascending=False).head(5)

,serial,player_name,club,position,goals,right_foot,left_foot,headers,others,inside_area,outside_areas,penalties,match_played
0,1,Benzema,Real Madrid,Forward,15,11,1,3,0,13,2,3,12
1,2,Lewandowski,Bayern,Forward,13,8,3,1,1,13,0,3,10
2,3,Haller,Ajax,Forward,11,3,4,3,1,11,0,1,8
3,4,Salah,Liverpool,Forward,8,0,8,0,0,7,1,1,13
4,5,Nkunku,Leipzig,Midfielder,7,3,1,3,0,7,0,0,6


**Вывод**: Хозяином штрафной можно назвать Lewandowski: 13 голов за 13 матчей.

In [71]:
goals.sort_values('penalties', ascending=False).head(5)

,serial,player_name,club,position,goals,right_foot,left_foot,headers,others,inside_area,outside_areas,penalties,match_played
0,1,Benzema,Real Madrid,Forward,15,11,1,3,0,13,2,3,12
1,2,Lewandowski,Bayern,Forward,13,8,3,1,1,13,0,3,10
5,5,Mahrez,Man. City,Midfielder,7,1,4,2,0,7,0,2,12
14,12,Messi,Paris,Forward,5,0,5,0,0,3,2,2,7
2,3,Haller,Ajax,Forward,11,3,4,3,1,11,0,1,8


**Вывод**: Аналогично исследованиям выше больше всего забитых пенальтей у лучших бомбардиров турнира: Benzema, Lewandowski

## Точность пассов

In [72]:
distr = pd.read_csv('distributon.csv')
distr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 608 entries, 0 to 607
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   serial           608 non-null    int64  
 1   player_name      608 non-null    object 
 2   club             608 non-null    object 
 3   position         608 non-null    object 
 4   pass_accuracy    608 non-null    float64
 5   pass_attempted   608 non-null    int64  
 6   pass_completed   608 non-null    int64  
 7   cross_accuracy   608 non-null    int64  
 8   cross_attempted  608 non-null    int64  
 9   cross_complted   608 non-null    int64  
 10  freekicks_taken  608 non-null    int64  
 11  match_played     608 non-null    int64  
dtypes: float64(1), int64(8), object(3)
memory usage: 57.1+ KB


In [73]:
distr = distr.query('match_played > 5')
len(distr)

318

In [74]:
distr.sort_values('pass_accuracy', ascending=False).head(5)

,serial,player_name,club,position,pass_accuracy,pass_attempted,pass_completed,cross_accuracy,cross_attempted,cross_complted,freekicks_taken,match_played
0,1,Erokhin,Zenit,Midfielder,98.0,27,26,0,2,0,0,6
4,5,J. Timber,Ajax,Midfielder,95.8,497,474,0,1,0,14,8
8,9,Süle,Bayern,Defender,94.7,429,413,0,2,0,9,7
9,9,Laporte,Man. City,Defender,94.7,707,672,0,0,0,10,9
10,11,Maycon,Shakhtar Donetsk,Midfielder,94.5,395,373,58,12,7,9,6


Неожиданно выборку возглавил Ерохин. Это связано с тем, что у него много сыгранных матчей, но очень мало игрового времени(игрок замены). Необходимо сделать доп. фильтрацию

In [75]:
distr = distr.query('pass_attempted	 > 350')
distr.sort_values('pass_accuracy', ascending=False).head(5)

,serial,player_name,club,position,pass_accuracy,pass_attempted,pass_completed,cross_accuracy,cross_attempted,cross_complted,freekicks_taken,match_played
4,5,J. Timber,Ajax,Midfielder,95.8,497,474,0,1,0,14,8
9,9,Laporte,Man. City,Defender,94.7,707,672,0,0,0,10,9
8,9,Süle,Bayern,Defender,94.7,429,413,0,2,0,9,7
10,11,Maycon,Shakhtar Donetsk,Midfielder,94.5,395,373,58,12,7,9,6
13,13,Stones,Man. City,Defender,94.4,545,523,0,1,0,8,8


Топ по точности пассов ожидаемо возглавили защитники. Как правило их пасы не несут остроты, отсюда и высокие показатели. Сделаем дополнительные топы для полузащтников и нападающих.

In [76]:
midf = distr.query('position	== "Midfielder"')
midf.sort_values('pass_accuracy', ascending=False).head(5)

,serial,player_name,club,position,pass_accuracy,pass_attempted,pass_completed,cross_accuracy,cross_attempted,cross_complted,freekicks_taken,match_played
4,5,J. Timber,Ajax,Midfielder,95.8,497,474,0,1,0,14,8
10,11,Maycon,Shakhtar Donetsk,Midfielder,94.5,395,373,58,12,7,9,6
15,16,Rodri,Man. City,Midfielder,94.2,806,760,0,1,0,13,10
29,29,Álvarez,Ajax,Midfielder,93.3,363,340,0,1,0,7,7
52,53,Kroos,Real Madrid,Midfielder,91.9,814,752,20,34,7,44,12


**Вывод**: Лучшие по показателю точности пассов среди полузащитников являются - Rodri и Kroos (c учётом выборки)

In [77]:
at = distr.query('position	== "Forward"')
at.sort_values('pass_accuracy', ascending=False).head(5)

,serial,player_name,club,position,pass_accuracy,pass_attempted,pass_completed,cross_accuracy,cross_attempted,cross_complted,freekicks_taken,match_played
104,104,Messi,Paris,Forward,89.9,428,385,35,14,5,12,7
303,304,Benzema,Real Madrid,Forward,83.1,523,433,20,10,2,4,12
304,304,Vinícius Júnior,Real Madrid,Forward,83.1,451,377,31,19,6,0,13
352,349,Salah,Liverpool,Forward,81.2,426,346,9,11,1,1,13
407,408,Müller,Bayern,Forward,78.9,366,283,28,25,7,1,10


**Вывод**: Лучшие по показателю точности пассов среди форвардов являются - Benzema и Messi

### Фолы

In [78]:
disс = pd.read_csv('disciplinary.csv')
disс.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 584 entries, 0 to 583
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   serial           584 non-null    int64 
 1   player_name      584 non-null    object
 2   club             584 non-null    object
 3   position         584 non-null    object
 4   fouls_committed  584 non-null    int64 
 5   fouls_suffered   584 non-null    int64 
 6   red              584 non-null    int64 
 7   yellow           584 non-null    int64 
 8   minutes_played   584 non-null    int64 
 9   match_played     584 non-null    int64 
dtypes: int64(7), object(3)
memory usage: 45.8+ KB


In [79]:
disс = disс.query('minutes_played > 350')
len(disс)

291

In [80]:
disс['fols_rate'] = disс['minutes_played'] / disс['fouls_committed']
disс.sort_values('fols_rate', ascending=True).head(5)

,serial,player_name,club,position,fouls_committed,fouls_suffered,red,yellow,minutes_played,match_played,fols_rate
4,5,Balanta,Club Brugge,Midfielder,18,3,4,0,407,5,22.611111
0,1,João Palhinha,Sporting CP,Midfielder,23,17,2,0,534,6,23.217391
14,12,Gavi,Barcelona,Midfielder,16,7,2,0,398,6,24.875000
35,31,Acuña,Sevilla,Defender,12,6,0,0,379,5,31.583333
17,16,Keïta,Liverpool,Midfielder,15,7,0,0,487,10,32.466667


**Вывод**: Balanta, João Palhinha и Gavi игроки которые нарушали правила чаще всего. Не реже 1 раза за 25 минут игры.|

### Удары по воротам

In [81]:
att = pd.read_csv('attempts.csv')
att.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 546 entries, 0 to 545
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   serial          546 non-null    int64 
 1   player_name     546 non-null    object
 2   club            546 non-null    object
 3   position        546 non-null    object
 4   total_attempts  546 non-null    int64 
 5   on_target       546 non-null    int64 
 6   off_target      546 non-null    int64 
 7   blocked         546 non-null    int64 
 8   match_played    546 non-null    int64 
dtypes: int64(6), object(3)
memory usage: 38.5+ KB


In [82]:
att = att.query('match_played > 5')
len(att)

288

Описание столбцов:  
total_attempts - всего ударов  
on_target - ударов в створ  
off_target - ударов мимо  
blocked - заблокированных ударов  

In [83]:
att.groupby('position')[['total_attempts', 'on_target']].agg(['count', 'mean'])

total_attempts            on_target          
                    count       mean     count      mean
position                                                
Defender               90   4.288889        90  1.166667
Forward                68  14.220588        68  5.808824
Midfielder            130   7.215385       130  2.507692

**Вывод:** чаще всего по воротам бьют форварды, реже всего защитники. Также защитники реже всего попадают в створ ворот.

In [84]:
att['att_rate'] = att['total_attempts'] / att['match_played']
att['att_target'] = att['on_target'] / att['match_played']
att['att_accur'] = att['on_target'] / att['total_attempts']

In [85]:
att.sort_values('att_rate', ascending=False).head(5)

,serial,player_name,club,position,total_attempts,on_target,off_target,blocked,match_played,att_rate,att_target,att_accur
0,1,Benzema,Real Madrid,Forward,45,23,13,9,12,3.750000,1.916667,0.511111
11,10,Messi,Paris,Forward,26,10,8,8,7,3.714286,1.428571,0.384615
10,10,Antony,Ajax,Forward,26,10,9,7,7,3.714286,1.428571,0.384615
20,19,Zapata,Atalanta,Forward,22,8,7,7,6,3.666667,1.333333,0.363636
14,13,Burak Yilmaz,LOSC,Forward,25,8,9,8,7,3.571429,1.142857,0.320000


**Вывод**: Лучшие по количеству ударов за матч: Benzema, Messi, Antony

In [86]:
att.sort_values('att_target', ascending=False).head(5)

,serial,player_name,club,position,total_attempts,on_target,off_target,blocked,match_played,att_rate,att_target,att_accur
7,8,Mbappé,Paris,Forward,27,16,6,5,8,3.375000,2.000000,0.592593
0,1,Benzema,Real Madrid,Forward,45,23,13,9,12,3.750000,1.916667,0.511111
4,5,Lewandowski,Bayern,Forward,31,19,8,4,10,3.100000,1.900000,0.612903
15,16,Haller,Ajax,Forward,24,15,8,1,8,3.000000,1.875000,0.625000
27,28,Nkunku,Leipzig,Midfielder,17,11,4,2,6,2.833333,1.833333,0.647059


**Вывод**: Лучшие по количеству ударов в створ за матч: Mbappé, Benzema, Lewandowski. В среднем они делают 2 удара в створ за матч.

In [87]:
att = att.query('total_attempts > 9')
att.sort_values('att_accur', ascending=False).head(5)

,serial,player_name,club,position,total_attempts,on_target,off_target,blocked,match_played,att_rate,att_target,att_accur
27,28,Nkunku,Leipzig,Midfielder,17,11,4,2,6,2.833333,1.833333,0.647059
30,31,Adeyemi,Salzburg,Forward,16,10,3,3,8,2.000000,1.250000,0.625000
15,16,Haller,Ajax,Forward,24,15,8,1,8,3.000000,1.875000,0.625000
4,5,Lewandowski,Bayern,Forward,31,19,8,4,10,3.100000,1.900000,0.612903
36,36,Reus,Dortmund,Midfielder,15,9,3,3,6,2.500000,1.500000,0.600000


**Вывод**: Лучшие по точности ударов: Nkunku, Adeyemi, Haller, Lewandowski, Reus.  
Более 60% от их ударов по воротам приходятся в створ.

## Символическая сборная

На основании проведенного анализа можно сформировать следующий состав символической сборной.

**Вратарь:** Courtois (Vlachodimos)  
 
**Защита:** Reinildo, Thiago Silva, De Ligt, Aké  

**Опорный полузащитник:**  Fernando(Lauper), Kross(Rodri) 

**Полузащита**: Sane(Vinícius Júnior), Bruno(Mbappe), Messi(Coman)

**Нападение**: Benzema(Levandovski)